##Exemplo Classificador de Flores - TensorFlow
 

(iris.jpg)

In [4]:
import tensorflow as tf
import pandas as pd

COLUMN_NAMES = [
        'SepalLength', 
        'SepalWidth',
        'PetalLength', 
        'PetalWidth', 
        'Species'
        ]

# Importando dataset de treinamento
training_dataset = pd.read_csv('iris_training.csv', names=COLUMN_NAMES, header=0)
train_x = training_dataset.iloc[:, 0:4]
train_y = training_dataset.iloc[:, 4]

# Importando dataset de teste
test_dataset = pd.read_csv('iris_test.csv', names=COLUMN_NAMES, header=0)
test_x = test_dataset.iloc[:, 0:4]
test_y = test_dataset.iloc[:, 4]


(iris_dataset.png)

In [5]:
# Agora, precisamos definir colunas de recursos, que irão ajudar a nossa Rede Neural.

columns_feat = [

    tf.feature_column.numeric_column(key='SepalLength'),
    tf.feature_column.numeric_column(key='SepalWidth'),
    tf.feature_column.numeric_column(key='PetalLength'),
    tf.feature_column.numeric_column(key='PetalWidth')
]

(iris_modelo.png)

In [6]:
#Construindo a Rede Neural - DNNClassifier

classifier = tf.estimator.DNNClassifier(

    feature_columns=columns_feat,

    # 2 camadas ocultas com 10 nós cada
    hidden_units=[10, 10],

    # O modelo está classificando 3 classes
    n_classes=3)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x118432e50>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/j4/f5p_3xrn2n12xxlnz1t9p2q40000gn/T/tmpC8gSQq', '_save_summary_steps': 100}


In [7]:
#Definindo função de treinamento

def train_function(inputs, outputs, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(inputs), outputs))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()


#Treinar o Modelo

classifier.train(
    input_fn=lambda:train_function(train_x, train_y, 100),
    steps=1000)


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/j4/f5p_3xrn2n12xxlnz1t9p2q40000gn/T/tmpC8gSQq/model.ckpt.
INFO:tensorflow:loss = 182.60324, step = 1
INFO:tensorflow:global_step/sec: 437.98
INFO:tensorflow:loss = 17.339533, step = 101 (0.230 sec)
INFO:tensorflow:global_step/sec: 550.118
INFO:tensorflow:loss = 11.880699, step = 201 (0.179 sec)
INFO:tensorflow:global_step/sec: 565.151
INFO:tensorflow:loss = 5.868081, step = 301 (0.177 sec)
INFO:tensorflow:global_step/sec: 575.897
INFO:tensorflow:loss = 8.438668, step = 401 (0.176 sec)
INFO:tensorflow:global_step/sec: 560.469
INFO:tensorflow:loss = 6.879751, step = 501 (0.176 sec)
INFO:tensorflow:global_step/sec: 565.064
INFO:tensorflow:loss = 6.683238, step = 601 (0.177 sec)
INFO:tensorflow:global_step/sec: 559.084
INFO:tensorflow:loss = 4.5398417, step = 701 (0.179 sec)
INFO:tensorflow:global_step/sec: 565.617
INFO:tensorflow:loss = 4.9797, step = 801 (0.178 sec)
INFO:tensorflow:glob

In [8]:
#Definindo função de avaliação
def evaluation_function(attributes, classes, batch_size):
    attributes=dict(attributes)
    if classes is None:
        inputs = attributes
    else:
        inputs = (attributes, classes)
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

# Avaliando o classificador
eval_result = classifier.evaluate(
    input_fn=lambda:evaluation_function(test_x, test_y, 100))

INFO:tensorflow:Starting evaluation at 2018-02-06-16:59:36
INFO:tensorflow:Restoring parameters from /var/folders/j4/f5p_3xrn2n12xxlnz1t9p2q40000gn/T/tmpC8gSQq/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-02-06-16:59:37
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.96666664, average_loss = 0.06028107, global_step = 1000, loss = 1.8084321
